<a href="https://colab.research.google.com/github/praveenkumarbalakrishnan/tasks/blob/main/SIG731_Data_Wrangling_Task8HD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Title:** Task 8HD: SIG731-Data Wrangling

**Name:** Praveenkumar Balakrishnan

**Student Number:** 223029369

**Email Address:** s223029369@deakin.edu.au


# Import statements

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
import argparse
import re
import os
import time
import xml.etree.cElementTree as cetree

import pandas as pd

# Tasks

Selected Movies & TV dataset for this analysis

**Task 1**

Convert all the data tables (Badges, Comments, PostHistory, PostLinks, Posts, Tags, Users, Votes)
from XML to CSV, using custom code that you write yourself. Ideally, you should write a Python
function that takes a single input file name (.xml) and output file name (.csv) and performs the
conversion of a single dataset.


Column names for 8 dataset based on the schema documentation

In [129]:
file_path = '/content/drive/MyDrive/Inputs/'

badges_cols = ['Id', 'UserId', 'Name', 'Date', 'Class', 'TagBased']
comments_cols = [
    'Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserDisplayName',
    'UserId', 'ContentLicense']
post_history_cols = [
    'Id', 'PostHistoryTypeId', 'PostId', 'RevisionGUID', 'CreationDate',
    'UserId', 'UserDisplayName', 'Comment', 'Text', 'ContentLicense']
post_links_cols = [
    'Id', 'CreationDate', 'PostId', 'RelatedPostId', 'LinkTypeId']
posts_cols = [
    'Id', 'PostTypeId', 'AcceptedAnswerId', 'ParentId', 'CreationDate',
    'DeletionDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId',
    'OwnerDisplayName', 'LastEditorUserId', 'LastEditorDisplayName',
    'LastEditDate', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount',
    'CommentCount', 'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate',
    'ContentLicense']
tags_cols = [
    'Id', 'TagName', 'Count', 'ExcerptPostId', 'WikiPostId',
    'IsModeratorOnly', 'IsRequired']
users_cols = [
    'Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
    'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
    'ProfileImageUrl', 'AccountId']
votes_cols = [
    'Id', 'PostId', 'VoteTypeId', 'UserId',  'CreationDate', 'BountyAmount']

In [30]:
os.path.join('/content/drive/MyDrive/Inputs/', 'posts' + '.xml')

'/content/drive/MyDrive/Inputs/posts.xml'

In [127]:
def get_data(xml_file_name, cols_list):
    """
    This generator is to parse the xml data and yield as a row

    Parameters
    ----------
    file_name: str
      Input xml file name
    cols: list
      Column names of the input files
    
    Yield
    ------
    row: str
      Row from xml data
    """
    col_names = ''
    for col in cols_list:
      col_names = col_names + col + '|'
    col_names = col_names[:-1] + '\n'
    yield col_names

    for event, elem in cetree.iterparse(xml_file_name):
      row = ''
      if elem.tag == "row":
        for col in cols_list:
          if col in elem.attrib:
              print(elem.attrib.keys())
              val = elem.attrib[col].replace("|", " ").replace('\n', ' ')
              if col in ['Tags']:
                val = re.sub(r'<', '', val)
                val = re.sub(r'>', ' ', val)
              else:
                val = re.sub(r'<.?>', '', val)
                val = re.sub('[^A-Za-z0-9]+', ' ', val)
                val = re.sub(' +', ' ', val)
              row = row + val.rstrip(' p') + '|'
          else:
              row = row + '|'
        row = row[:-1] + '\n'
        yield row
        elem.clear()

def convert_xml_to_csv(file_name, cols_list, path):
  """
  This function is to convert xml to csv and write as csv file
  and upper value

  Parameters
  ----------
  file_name: str
    Input filename without the extension like ".xml"
  cols: list
    Column names of the input files
  
  Return
  ------
  csv_filename: str
    Output csv filename
  """
  xml_file_name = os.path.join(path, file_name + '.xml')
  output_file_name = os.path.join(path, file_name + '.csv')

  data = get_data(xml_file_name, cols_list)
  with open(output_file_name, 'w') as f:
      for item in data:
          f.write(item)
  return output_file_name

In [128]:
convert_xml_to_csv('Votes', posts_cols, file_path)

Streaming output truncated to the last 5000 lines.
dict_keys(['Id', 'PostId', 'VoteTypeId', 'CreationDate'])
dict_keys(['Id', 'PostId', 'VoteTypeId', 'CreationDate'])
dict_keys(['Id', 'PostId', 'VoteTypeId', 'CreationDate'])
dict_keys(['Id', 'PostId', 'VoteTypeId', 'CreationDate'])
dict_keys(['Id', 'PostId', 'VoteTypeId', 'CreationDate'])
dict_keys(['Id', 'PostId', 'VoteTypeId', 'CreationDate'])
dict_keys(['Id', 'PostId', 'VoteTypeId', 'CreationDate'])
dict_keys(['Id', 'PostId', 'VoteTypeId', 'CreationDate'])
dict_keys(['Id', 'PostId', 'VoteTypeId', 'CreationDate'])
dict_keys(['Id', 'PostId', 'VoteTypeId', 'CreationDate'])
dict_keys(['Id', 'PostId', 'VoteTypeId', 'CreationDate'])
dict_keys(['Id', 'PostId', 'VoteTypeId', 'CreationDate'])
dict_keys(['Id', 'PostId', 'VoteTypeId', 'CreationDate'])
dict_keys(['Id', 'PostId', 'VoteTypeId', 'CreationDate'])
dict_keys(['Id', 'PostId', 'VoteTypeId', 'CreationDate'])
dict_keys(['Id', 'PostId', 'VoteTypeId', 'CreationDate'])
dict_keys(['Id', 'Pos

KeyboardInterrupt: ignored

In [118]:
df = pd.read_csv('/content/drive/MyDrive/Inputs/Posts.csv', delimiter='|')
df

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,OwnerUserId,...,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,ContentLicense
0,1,1,2776.0,NaN,2011 11 30T19 15 54 070,NaN,31,8094.0,Some comedians actors are given creative freed...,11.0,...,2022 02 12T21 59 39 633,2022 02 12T21 59 39 633,To what extent were the actors in Wedding Cras...,wedding-crashers,2.0,0,NaN,NaN,NaN,CC BY SA 4 0
1,2,2,NaN,1.0,2011 11 30T19 37 10 510,NaN,15,NaN,According to the a href http www imdb com titl...,31.0,...,NaN,2011 11 30T19 37 10 510,NaN,NaN,NaN,3,NaN,NaN,NaN,CC BY SA 3 0
2,3,1,814.0,NaN,2011 11 30T19 41 14 960,NaN,30,1934.0,In his Star Wars Episode 1 a href https redlet...,41.0,...,2019 05 15T02 23 45 307,2019 05 15T02 23 45 307,Evaluating characters by means of description,analysis star-wars,2.0,2,NaN,NaN,NaN,CC BY SA 4 0
3,4,1,120.0,NaN,2011 11 30T19 42 45 470,NaN,59,3784.0,I m a big fan of the Pink Panther movies the o...,22.0,...,2013 08 17T20 17 23 187,2020 11 18T17 05 17 490,Was the original Pink Panther movie intended t...,comedy the-pink-panther,5.0,2,NaN,NaN,NaN,CC BY SA 3 0
4,6,1,21.0,NaN,2011 11 30T19 44 55 593,NaN,16,8239.0,At the end of the movie adult Jack Sean Penn w...,34.0,...,2013 06 06T12 05 01 613,2013 06 06T20 02 04 800,What does the ending of The Tree of Life mean,plot-explanation analysis ending the-tree-of-life,2.0,0,NaN,NaN,NaN,CC BY SA 3 0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62658,119111,2,NaN,40884.0,2022 12 04T10 01 04 300,NaN,0,NaN,In my opinion I believe the only reason Walt b...,98469.0,...,2022 12 04T12 24 03 370,2022 12 04T12 24 03 370,NaN,NaN,NaN,1,NaN,NaN,NaN,CC BY SA 4 0
62659,119112,1,NaN,NaN,2022 12 04T10 15 41 907,NaN,3,15.0,Does anyone know who the voice actor is that p...,98470.0,...,2022 12 04T10 36 36 783,2022 12 04T10 36 36 783,Who is the actor that does the voice for Galad...,voice-acting gary-and-his-demons,0.0,2,NaN,2022 12 04T14 50 46 017,NaN,CC BY SA 4 0
62660,119113,2,NaN,112491.0,2022 12 04T15 34 30 910,NaN,0,NaN,He was trying to say that when in the frontlin...,98475.0,...,2022 12 04T15 37 07 917,2022 12 04T15 37 07 917,NaN,NaN,NaN,0,NaN,NaN,NaN,CC BY SA 4 0
62661,119114,1,NaN,NaN,2022 12 04T18 21 06 160,NaN,0,6.0,I was just watching a href https www imdb com ...,12878.0,...,NaN,2022 12 04T18 21 06 160,Devo mentioned in Hot Shots 91,reference comedy,0.0,1,NaN,NaN,NaN,CC BY SA 4 0


In [44]:
df['Body'].values

array(['Some comedians actors are given creative freedom to improvise at times when producing a new film Is there any evidence that Vince Vaughn or Owen Wilson improvised in any scenes diverging from the script in the film quot Wedding Crashers quot',
       'According to the trivia on IMDb Owen Wilson and Vince Vaughn improvised the Lock it up banter As I understand it that also means the other scenes did not or only slightly diverge from the script',
       'In his Star Wars Episode 1 review analysis Mike Stoklasa used a certain challenge as a device to illustrate how poorly thought out the characters were Describe the following characters without mentioning what they look like what kind of costume they wore or what their profession or role in the movie was Describe them like you would to people who ve never seen Star Wars He then listed Han Solo Qui Gon Jin C3PO and Queen Amidala each followed by descriptions or attempts at them by various people Is this a correct way of assessing c

In [17]:
from csv import reader
# open file in read mode
with open('/content/drive/MyDrive/Inputs/test.csv', 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    count = 0
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
        count += 1
        # row variable is a list that represents a row in csv
        if count == 2 or count == 4:
          print(row, count)


['1|1|2776||2011 11 30T19 15 54 070||31|8094|Some comedians actors are given creative freedom to improvise at times when producing a new film Is there any evidence that Vince Vaughn or Owen Wilson improvised in any scenes diverging from the script in the film quot Wedding Crashers quot|11||94442||2022 02 12T21 59 39 633|2022 02 12T21 59 39 633|To what extent were the actors in Wedding Crashers improvising||2|0||||CC BY SA 4 0'] 2
['3|1|814||2011 11 30T19 41 14 960||30|1934|In his Star Wars Episode 1 review analysis Mike Stoklasa used a certain challenge as a device to illustrate how poorly thought out the characters were Describe the following characters without mentioning what they look like what kind of costume they wore or what their profession or role in the movie was Describe them like you would to people who ve never seen Star Wars He then listed Han Solo Qui Gon Jin C3PO and Queen Amidala each followed by descriptions or attempts at them by various people Is this a correct way o

In [ ]:
pd.read_csv('/content/drive/MyDrive/Inputs/tmp.csv')

ParserError: ignored